# Importing Dependencies 

In [56]:
import numpy as np
import os

import cv2
import matplotlib.pylab as plt
import imghdr

import tensorflow  as tf

In [57]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [64]:
data_dir = 'imgdata_v2/orginal_dataset/'
os.listdir(data_dir)

['cataract', 'normal']

In [65]:
for img_class in os.listdir(data_dir):
    print(img_class,': ' , len(os.listdir(os.path.join(data_dir, img_class))))

cataract :  1038
normal :  1074


In [66]:
img_exts = ['jpg', 'png', 'jpeg']
img_exts

['jpg', 'png', 'jpeg']

In [67]:
img_removed=0

for img_class in os.listdir(data_dir):
    for img in os.listdir(os.path.join(data_dir, img_class)):
        img_path = os.path.join(data_dir, img_class, img)
        img_ext  = imghdr.what(img_path)
        try:
            if img_ext not in img_exts:
                os.remove(img_path)
                # print('Removed {}'.format(img_path))
                img_removed+=1
        except Exception as e:
            print('Issue with {}'.format(img_path))

print('Delted ', img_removed, ' images from directory.')

Delted  0  images from directory.


In [68]:
import splitfolders

splitfolders.ratio(data_dir, 
                   output='imgdata_v2/split_dataset/',
                   seed=26,
                   ratio=(.7, .2, .1),
                   group_prefix=None)

Copying files: 2112 files [00:02, 759.64 files/s]


In [77]:
split_path = 'imgdata_v2/split_dataset/'
for split in os.listdir(split_path):
    for img_class in os.listdir(os.path.join(split_path, split)):
        print(img_class, split, ': ', len(os.listdir(os.path.join(split_path, split, img_class))))
    print()

cataract test :  105
normal test :  109

cataract train :  726
normal train :  751

cataract val :  207
normal val :  214



In [80]:
train_dir = 'imgdata_v2/split_dataset/train/'
test_dir = 'imgdata_v2/split_dataset/test/'
val_dir = 'imgdata_v2/split_dataset/val/'

In [140]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(brightness_range=(0.1, 1.3),
                            horizontal_flip=True,
                            vertical_flip=True,
                            fill_mode='nearest')

In [93]:
train_datagen.flow_from_directory??

Signature:
train_datagen.flow_from_directory(
    directory,
    target_size=(256, 256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    subset=None,
    interpolation='nearest',
    keep_aspect_ratio=False,
)
Source:   
    def flow_from_directory(
        self,
        directory,
        target_size=(256, 256),
        color_mode="rgb",
        classes=None,
        class_mode="categorical",
        batch_size=32,
        shuffle=True,
        seed=None,
        save_to_dir=None,
        save_prefix="",
        save_format="png",
        follow_links=False,
        subset=None,
        interpolation="nearest",
        keep_aspect_ratio=False,
    ):
        """Takes the path to a directory & generates batches of augmented data.

        Args:
            directory: string, path to the target directory. It should contain

In [141]:
augmented_dir = 'imgdata_v2/split_dataset/train/augmented/'

# for img_class in os.listdir(train_dir):
#     for img in os.listdir(os.path.join(train_dir, img_class)):
        # img_path = os.path.join(train_dir, img_class, img)
img_path = 'imgdata_v2/split_dataset/train/cataract/_1_5346540.jpg'
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_reshaped = img.reshape((1,)+img.shape)

i=1
for batch in train_datagen.flow(img_reshaped,
                         batch_size=1,
                         save_to_dir=os.path.join(augmented_dir),
                         save_prefix='DA',
                         save_format='jpg'):
    i+=1
    if i>5: break 